In [ ]:
import os
import json
import pandas as pd
from pathlib import Path

In [ ]:
BASE_PATH = Path(os.path.abspath('')).parent.absolute()

In [ ]:
f = open(str(BASE_PATH)+'\\data\\states.json')
indian_states = json.load(f)

f = open(str(BASE_PATH)+'\\data\\state_district_wise.json')
indian_covid_json  = json.load(f)

In [ ]:
# JSON of Covid Data to pd (index=district)
state_pd = []

for state in indian_states['features']:
    
    curr_state = state['id']
    
    current_state_pd = pd.DataFrame(indian_covid_json[curr_state]['districtData']).T
    
    for status in current_state_pd.iloc[0,:]['delta'].keys():
        current_state_pd[f'delta{status.title()}'] = current_state_pd['delta'].apply(lambda x: x[status])
        
    current_state_pd.drop(columns=['delta'], inplace=True)
    
    current_state_pd['state'] = curr_state
    
    state_pd.append(current_state_pd)
    
full_data = pd.concat(state_pd)

In [ ]:
full_data = full_data.astype(dtype={
    "notes":"object",
    "active":"int64",
    "confirmed":"int64",
    "migratedother":"int64",
    "deceased":"int64",
    "recovered":"int64",
    "deltaConfirmed":"int64",
    "deltaDeceased":"int64",
    "deltaRecovered":"int64",
    "state":"object"
})

In [ ]:
full_data.dtypes

In [ ]:
covid_state_data = (
    full_data
    .groupby(by='state')
    .sum(numeric_only=True)
)


In [ ]:
geo_data_pd = []

for state in indian_states['features']:
    geo_data_pd.append(pd.DataFrame(pd.json_normalize(state)))
    
full_geo_data = (pd.concat(geo_data_pd)
                 .drop(columns=['type','total'])
                 .rename(columns={'id' : 'state'})
                 .set_index('state'))    

In [ ]:
covid_geo_pd = (covid_state_data
                    .merge(full_geo_data, on='state')
                    .reset_index()
                    .rename(columns={'state' : 'id'})
                    .to_json(orient='table', index=False)
               )

In [ ]:
with open('covid_geo_india.json','w') as file:
    file.write(covid_geo_pd)

In [ ]:
(covid_geo_pd
)

In [ ]:
import geojson

In [ ]:
with open(str(BASE_PATH)+'\\data\\states.json') as f:
    feat_collection = geojson.load(f)

In [ ]:
state_properties = covid_state_data.T.to_dict()

In [ ]:
for i,state in enumerate(feat_collection["features"]):
    if state['id'] in state_properties.keys():
        feat_collection["features"][i]['properties'] = state_properties[state['id']]

In [ ]:
feat_collection["features"][0]['properties']

In [57]:
with open('covid_geo_india.json','w') as file:
    geojson.dump(feat_collection, file)